In [ ]:
# @title Connect to gdrive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
#@title Download a file
import os
def download_file(url):
    !wget -O {url}  {os.path.basename(url)}
url = input("Enter Url:          ")
download_file(url)

In [ ]:
#@title Download Muliple file
import os

def download_file(url, i):
    os.system(f"wget -O seg-{i}-v1-a1.ts {url}")

for i in range(61, 91):
    download_file(f"https://58a49ee706238.streamlock.net:8485/uploads/jio_271144/1f3ac880567e7e84612e04d3d5186efc.mp4/seg-{i}-v1-a1.ts", i)


In [ ]:
#@title Download file oldway
!wget -O "/content/drive/MyDrive/10.mkv"  ""

--2023-12-15 23:13:59--  https://scontent.fdac24-3.fna.fbcdn.net/v/t42.1790-2/10000000_383361337478713_5138757873228744901_n.mp4?_nc_cat=104&ccb=1-7&_nc_sid=55d0d3&efg=eyJybHIiOjM0NSwicmxhIjo0MDk2LCJ2ZW5jb2RlX3RhZyI6InN2ZV9zZCJ9&_nc_eui2=AeEbjEU3FIv-lX7Ca455mUbJqwBZ0q0pcnerAFnSrSlyd2WtPI79D3f_IDGTy-KtxcJ8M-DydZ_cFEGKv7xzg0b0&_nc_ohc=-pqaq9eoJ1AAX_OzvgJ&_nc_rml=0&_nc_ht=scontent.fdac24-3.fna&oh=00_AfCtELeY4W-IAq46ccB1cur2Fe8u2y_TaA_0FiHY78l9UA&oe=6581E215
Resolving scontent.fdac24-3.fna.fbcdn.net (scontent.fdac24-3.fna.fbcdn.net)... 103.242.216.19, 2405:1500:0:12:face:b00c:0:a7
Connecting to scontent.fdac24-3.fna.fbcdn.net (scontent.fdac24-3.fna.fbcdn.net)|103.242.216.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66150985 (63M) [video/mp4]
Saving to: ‘/content/drive/MyDrive/10.mkv’

                     70%[=============>      ]  44.52M  44.0KB/s    eta 41s    

In [5]:
# @title Unzip a file
!pip install patool
import patoolib
import os

def unzip_file(file_path, extract_path):
    try:
        patoolib.extract_archive(file_path, outdir=extract_path)
        print(f"Successfully extracted {file_path} to {extract_path}")
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")

# Example usage
compressed_file_path = '/content/drive/MyDrive/'+ input("Enter fileName in MyDrive:     ")
extract_path = '/content/drive/MyDrive/extrected'+input("Enter output folder in MyDrive:     ")

unzip_file(compressed_file_path, extract_path)
os.remove(compressed_file_path)
print(f"unzipped succesfully and deleted zip file")





Enter fileName in MyDrive:     z.zip


INFO patool: Extracting /content/drive/MyDrive/z.zip ...
INFO:patool:Extracting /content/drive/MyDrive/z.zip ...
INFO patool: running /usr/bin/7z x -o/content/drive/MyDrive/extrected -- /content/drive/MyDrive/z.zip
INFO:patool:running /usr/bin/7z x -o/content/drive/MyDrive/extrected -- /content/drive/MyDrive/z.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/drive/MyDrive/z.zip extracted to `/content/drive/MyDrive/extrected'.
INFO:patool:... /content/drive/MyDrive/z.zip extracted to `/content/drive/MyDrive/extrected'.


Successfully extracted /content/drive/MyDrive/z.zip to /content/drive/MyDrive/extrected
unzipped succesfully and deleted zip file


In [ ]:
# @title Extract audio track
#extract audio
import subprocess

def extract_audio(input_video, output_audio):
    command = [
        'ffmpeg',
        '-i', input_video,
        '-vn',  # Disable video recording
        '-acodec', 'copy',  # Copy audio codec without re-encoding
        #'-map', '0:a:1',  # Select the second audio stream (0-indexed)
        output_audio
    ]

    subprocess.run(command)

if __name__ == "__main__":
    input_video_file = "ben.mkv"
    output_audio_file = "ben2.mp4"
    for i in range(11,20):
        extract_audio(f"{i}.mkv", f"./nullvideo/{i}-vid.mp4")

In [ ]:
ls

In [ ]:
# @title Audio track count find
#audio track count find
import subprocess
import json

def get_all_audio_track_info(input_video):
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'a',
        '-show_entries', 'stream=index:stream_tags=language',
        '-of', 'json',
        input_video
    ]

    try:
        result = subprocess.run(command, capture_output=True, text=True)
        output = result.stdout

        data = json.loads(output)
        audio_tracks = [
            {"index": entry["index"], "language": entry["tags"].get("language", "unknown")}
            for entry in data.get("streams", [])
        ]

        return audio_tracks

    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    input_video_file = "ben.mkv"

    audio_track_info = get_all_audio_track_info(input_video_file)

    if audio_track_info is not None:
        print("All Audio Tracks Information:")
        for track in audio_track_info:
            print(f"Index: {track['index']}, Language: {track['language']}")
    else:
        print("Failed to retrieve audio track information.")

In [ ]:
# @title h264 video(aac) => h264 video(mp3)
import subprocess

def copy_video_change_audio_to_mp3(input_video, output_video):
    command = [
        'ffmpeg',
        '-i', input_video,
        '-c:v', 'copy',    # Copy the video codec without re-encoding
        '-c:a', 'mp3',     # Set the audio codec to MP3
        '-strict', 'experimental',  # Needed for using AAC codec
        output_video
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Video copied, and audio codec changed to MP3. Saved to {output_video}.")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    input_video_file = "path/to/your/input/video.mp4"
    output_video_file = "path/to/your/output/copied_video.mp4"
    for i in range(1,7):
        copy_video_change_audio_to_mp3(f"{i}.mkv", f"./mp3vid/{i}.mp4")


In [ ]:
#@title Zip a folder
import shutil

def zip_folder(folder_path, zip_path):
    # Create a zip file with the contents of the specified folder
    shutil.make_archive(zip_path, 'zip', folder_path)

# Example usage:
folder_to_zip = '/content/drive/MyDrive/'+input('Enter Folder MyDrive/           ')
zip_file_path = '/content/drive/MyDrive/folder.zip'

zip_folder(folder_to_zip, zip_file_path)


Enter Folder MyDrive/           mp3vid


In [ ]:
#@title Rename file
from google.colab import drive
import os

directory_path = '/content/drive/MyDrive/'
os.chdir(directory_path)

for i in range(1,2):
    os.rename(f"{i}.mp4", f"Mr. Bad EP-{i}.mp4")


FileNotFoundError: ignored

In [ ]:
fmpeg -i "16.mkv" -c:v libx264 -crf 32 -preset superfast -c:a mp3 -b:a 192k -threads 8 "16.mp4"

SyntaxError: ignored

In [ ]:
# Install FFmpeg
!apt-get install ffmpeg

# Run FFmpeg command
!ffmpeg -i "16.mkv" -c:v libx264 -crf 32 -preset superfast -c:a mp3 -b:a 192k -threads 8 "16.mp4"




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -